## init

In [2]:
import sys
# print(sys.path)

!python --version

Python 3.12.7


In [3]:
!pip install -q mediapipe

In [4]:
# 모델 파일 다운로드 (Pose Landmarker Lite, Float16, v1)
!wget -O pose_landmarker.task -q https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task

'wget'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


## Visualization utilities

In [15]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

def draw_landmarks_on_image(rgb_image, detection_result):
    if not detection_result.pose_landmarks:
        return rgb_image

    # detection_result.pose_landmarks 가
    #  - 여러 명이면 list of NormalizedLandmarkList
    #  - 한 명이면 NormalizedLandmarkList
    raw = detection_result.pose_landmarks
    # 한 명일 때에는 list 로 감싸 주기
    if not isinstance(raw, (list, tuple)):
        pose_landmarks_list = [raw]
    else:
        pose_landmarks_list = raw

    annotated_image = np.copy(rgb_image)
    for pose_landmarks in pose_landmarks_list:
        # landmark_pb2 객체로 변환
        proto = landmark_pb2.NormalizedLandmarkList()
        proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=l.x, y=l.y, z=l.z)
            for l in pose_landmarks.landmark
        ])
        # 그리기
        solutions.drawing_utils.draw_landmarks(
            annotated_image,
            proto,
            solutions.pose.POSE_CONNECTIONS,
            solutions.drawing_styles.get_default_pose_landmarks_style()
        )
    return annotated_image


In [16]:
import cv2
import mediapipe as mp
import time
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

## 이미지로 MPP 동작

In [17]:
cap = cv2.VideoCapture(0)

with mp_pose.Pose(
    model_complexity=2, # 0: lite, 1: full, 2: heavy
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    results = pose.process(image)

    keypoints = []
    if results.pose_landmarks:
        for data_point in results.pose_landmarks.landmark:
            keypoints.append({
                'X': data_point.x,
                'Y': data_point.y,
                'Z': data_point.z,
                'Visibility': data_point.visibility,
            })

    # Draw the pose annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()

KeyboardInterrupt: 

## 영상으로 MPP 동작

### sit / stand로 구분

#### 스쿼트 횟수 카운트

In [ ]:
squat_videoPath = 'C:\\Users\\mia00\\Desktop\\CD\\resources\\squat.mp4'
cap = cv2.VideoCapture(squat_videoPath)

squat_count = 0  # 스쿼트 횟수
is_down = False  # 현재 자세가 내려간 상태인지 확인
initial_hip_y = None  # 처음 자세에서의 엉덩이 위치

with mp_pose.Pose(
    model_complexity=2,  # 0: lite, 1: full, 2: heavy
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Video has ended or failed to load.")
      break

    # BGR → RGB 변환
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    results = pose.process(image)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        # 오른쪽 엉덩이, 무릎, 발목의 Y 좌표 가져오기
        # right_hip_y = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y
        # right_knee_y = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y
        # right_ankle_y = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y

        # 왼쪽 엉덩이, 무릎, 발목의 Y 좌표 가져오기
        left_hip_y = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
        left_knee_y = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y
        left_ankle_y = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y

        # 디버깅: Y 좌표 출력
        # print(f"Hip Y: {right_hip_y}, Knee Y: {right_knee_y}, Ankle Y: {right_ankle_y}")
        # print(f"Left Hip Y: {left_hip_y}, Left Knee Y: {left_knee_y}, Left Ankle Y: {left_ankle_y}")
        # print(f"Left Hip Y: {left_hip_y}")

        # 처음 자세의 엉덩이 위치 저장
        if initial_hip_y is None:
            initial_hip_y = left_hip_y
            print(f"Initial Hip Y: {initial_hip_y}")

        # 스쿼트 동작 감지
        if abs(left_hip_y - left_knee_y) < 0.05:  # 엉덩이가 무릎과 비슷한 높이에 있을 때
            if not is_down:
                print("Down position detected")
                is_down = True
        elif is_down and left_hip_y < left_knee_y - 0.05 and (left_hip_y <= initial_hip_y + 0.05):  # 엉덩이가 무릎보다 위로 올라오고 처음 자세랑 비슷한 높이 이상일 때
            squat_count += 1
            is_down = False
            print(f"스쿼트 횟수: {squat_count}")

        # if right_hip_y > right_knee_y + 0.05 and right_knee_y > right_ankle_y + 0.05:  # 엉덩이가 무릎보다 아래로 내려갔을 때
        #     if not is_down:
        #         print("Down position detected")
        #         is_down = True
        # elif is_down and right_hip_y <= right_knee_y + 0.02:  # 다시 올라왔을 때
        #     squat_count += 1
        #     is_down = False
        #     print(f"스쿼트 횟수: {squat_count}")

    # Draw the pose annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

    # 스쿼트 횟수를 영상에 표시
    cv2.putText(
        image,
        f"Squat Count: {squat_count}",
        (50, 50),  # 텍스트 위치 (x, y)
        cv2.FONT_HERSHEY_SIMPLEX,  # 폰트
        1,  # 폰트 크기
        (0, 255, 0),  # 텍스트 색상 (초록색)
        2,  # 텍스트 두께
        cv2.LINE_AA  # 선형 보간
    )

    resized_image = cv2.resize(image, (720, 1080))  # 화면 크기 조정

    # Flip 제거 (원본 영상 그대로 표시)
    cv2.imshow('MediaPipe Pose', resized_image)
    if cv2.waitKey(5) & 0xFF == 27:
      break

cap.release()
cv2.destroyAllWindows()

Initial Hip Y: 0.5344144701957703


#### 무릎 굽혀짐 검사

In [ ]:
def calculate_angle(a, b, c):
    """
    세 점 a, b, c를 기준으로 b를 중심으로 한 각도를 계산합니다.
    a, b, c는 각각 (x, y) 좌표를 나타냅니다.
    """
    a = np.array(a)  # 첫 번째 점
    b = np.array(b)  # 중심점
    c = np.array(c)  # 세 번째 점

    # 벡터 계산
    ab = a - b
    bc = c - b

    # 벡터 사이의 각도 계산
    cosine_angle = np.dot(ab, bc) / (np.linalg.norm(ab) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))  # arccos의 입력값을 -1~1로 제한
    return np.degrees(angle)  # 라디안을 도(degree)로 변환

In [ ]:
squat_videoPath = 'C:\\Users\\mia00\\Desktop\\CD\\resources\\squat.mp4'
cap = cv2.VideoCapture(squat_videoPath)

squat_count = 0  # 스쿼트 횟수
is_down = False  # 현재 자세가 내려간 상태인지 확인
initial_hip_y = None  # 처음 자세에서의 엉덩이 위치

with mp_pose.Pose(
    model_complexity=2,  # 0: lite, 1: full, 2: heavy
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Video has ended or failed to load.")
      break

    # BGR → RGB 변환
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    results = pose.process(image)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        # 왼쪽 엉덩이, 무릎, 발목 좌표 가져오기
        left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
        left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
        left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

         # 무릎 각도 계산
        knee_angle = calculate_angle(left_hip, left_knee, left_ankle)

        # 무릎 펴짐 상태 확인
        if knee_angle > 160:
            status = "Knee Straight"
        else:
            status = "Knee Bent"

        # 무릎 각도와 상태를 화면에 표시
        cv2.putText(image, f"Knee Angle: {int(knee_angle)}", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(image, status, (50, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Draw the pose annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

    # 스쿼트 횟수를 영상에 표시
    cv2.putText(
        image,
        f"Squat Count: {squat_count}",
        (50, 150),  # 텍스트 위치 (x, y)
        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA
    )

    resized_image = cv2.resize(image, (720, 1080))  # 화면 크기 조정

    # Flip 제거 (원본 영상 그대로 표시)
    cv2.imshow('MediaPipe Pose', resized_image)
    if cv2.waitKey(5) & 0xFF == 27:
      break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
# from google.colab.patches import cv2_imshow
from mediapipe.framework.formats import landmark_pb2

cap = cv2.VideoCapture(0)
fps = cap.get(cv2.CAP_PROP_FPS) or 30.0

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    static_image_mode=False,           # 실시간 모드
    model_complexity=1,                # 모델 복잡도 (0~2)
    enable_segmentation=False,         # 세그멘테이션 사용 여부
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 타임스탬프(ms) 계산 (필요 시)
    timestamp_ms = int(frame_idx * 1000 / fps)
    frame_idx += 1

    # BGR→RGB
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    rgb.flags.writeable = False

    # ── 3) 포즈 검출 ──
    results = pose.process(rgb)

    # ── 4) 키포인트 추출 ──
    keypoints = []
    if results.pose_landmarks:
        for lm in results.pose_landmarks.landmark:
            keypoints.append({
                'X': lm.x,
                'Y': lm.y,
                'Z': lm.z,
                'Visibility': lm.visibility,
                'Timestamp_ms': timestamp_ms
            })
    # keypoints 리스트를 여기서 활용하세요

    # ── 5) 시각화 ──
    rgb.flags.writeable = True
    annotated = draw_landmarks_on_image(rgb, results)

    # RGB→BGR
    annotated_bgr = cv2.cvtColor(annotated, cv2.COLOR_RGB2BGR)
    # cv2_imshow(annotated_bgr)
    cv2.imshow('MediaPipe Pose', cv2.flip(annotated_bgr, 1))

    # 27키(ESC) 누르면 종료
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
pose.close()
cv2.destroyAllWindows()

### 초기 자세 / sit / stand 구분

#### 스쿼트 횟수 카운트

In [ ]:
import time

squat_videoPath = 'C:\\Users\\mia00\\Desktop\\CD\\resources\\squat.mp4'
cap = cv2.VideoCapture(squat_videoPath)

squat_count = 0  # 스쿼트 횟수
initial_hip_y = None  # 처음 자세에서의 엉덩이 위치

state = "START"  # 초기 상태
sit_start_time = None  # sit 상태 시작 시간
time_limit = 10  # 시간 제한 (초)
previous_hip_y = None  # 이전 프레임의 엉덩이 Y 좌표

with mp_pose.Pose(
    model_complexity=2,  # 0: lite, 1: full, 2: heavy
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Video has ended or failed to load.")
      break

    # BGR → RGB 변환
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    results = pose.process(image)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        # 왼쪽 엉덩이, 무릎, 발목, 어깨의 Y 좌표 가져오기
        left_hip_y = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
        left_knee_y = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y
        left_ankle_y = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y
        left_shoulder_y = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y

        # 처음 자세의 엉덩이 위치 저장
        if initial_hip_y is None:
            initial_hip_y = left_hip_y
            print(f"Initial Hip Y: {initial_hip_y}")
        # 허리 펴짐 정도 계산 (엉덩이와 어깨의 Y 좌표 차이)
        back_straightness = abs(left_hip_y - left_shoulder_y)

        # 상태 전환 로직
        if state == "START":
            if back_straightness < 0.1:  # 허리가 펴져 있는지 확인
                print("올바른 자세로 서 있음")
            if abs(left_hip_y - left_knee_y) < 0.05: # 엉덩이가 무릎과 비슷한 높이에 있을 때
                state = "SIT"
                sit_start_time = time.time()  # sit 상태 시작 시간 기록
                print("State changed to SIT")
        elif state == "SIT":
            if time.time() - sit_start_time > time_limit:  # 시간 초과
                state = "START"
                print("시간 초과! State changed to START")
            elif left_hip_y < left_knee_y - 0.05 and (left_hip_y <= initial_hip_y + 0.05): # 엉덩이가 무릎보다 위로 올라오고 처음 자세랑 비슷한 높이 이상일 때
                state = "STAND"
                squat_count += 1
                print(f"스쿼트 횟수: {squat_count}")
        elif state == "STAND" and left_hip_y >= left_knee_y - 0.05: # 다시 앉기 시작
            state = "SIT"
            sit_start_time = time.time()  # sit 상태 시작 시간 갱신
            print("State changed to SIT")

    # Draw the pose annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

    # 스쿼트 횟수를 영상에 표시
    cv2.putText(
        image,
        f"State: {state}, Squat Count: {squat_count}",
        (50, 50),  # 텍스트 위치 (x, y)
        cv2.FONT_HERSHEY_SIMPLEX,  # 폰트
        1,  # 폰트 크기
        (0, 255, 0),  # 텍스트 색상 (초록색)
        2,  # 텍스트 두께
        cv2.LINE_AA  # 선형 보간
    )
    # sit 상태에서 경과 시간 표시
    if state == "SIT" and sit_start_time is not None:
        elapsed_time = time.time() - sit_start_time
        cv2.putText(
            image,
            f"Time: {elapsed_time:.1f}s",
            (50, 100),  # 텍스트 위치 (x, y)
            cv2.FONT_HERSHEY_SIMPLEX,  # 폰트
            1,  # 폰트 크기
            (0, 255, 255),  # 텍스트 색상 (노란색)
            2,  # 텍스트 두께
            cv2.LINE_AA  # 선형 보간
        )

    resized_image = cv2.resize(image, (720, 1080))  # 화면 크기 조정

    # Flip 제거 (원본 영상 그대로 표시)
    cv2.imshow('MediaPipe Pose', resized_image)
    if cv2.waitKey(5) & 0xFF == 27:
      break

cap.release()
cv2.destroyAllWindows()

Initial Hip Y: 0.5344144701957703
State changed to SIT
스쿼트 횟수: 1
State changed to SIT
스쿼트 횟수: 2


## 실시간으로 MPP 동작

### 실시간

In [13]:
import math
import time
import cv2
import numpy as np
import mediapipe as mp

# MediaPipe 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# 스쿼트 영상 경로
# VIDEO_PATH = 'C:\\Users\\mia00\\Desktop\\CD\\resources\\squat.mp4'
MAX_WIDTH = 640  # 영상 최대 너비
TIME_LIMIT = 10  # 시간 제한 (초)
BACK_STRAIGHTNESS_THRESHOLD = 0.15  # 허리 펴짐 기준

In [14]:
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ab = a - b
    bc = c - b

    cosine_angle = np.dot(ab, bc) / (np.linalg.norm(ab) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

In [ ]:
def check_knee_angle(landmarks):
    left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

    # 무릎 각도 계산
    knee_angle = calculate_angle(left_hip, left_knee, left_ankle)

    # 피드백 생성
    if knee_angle < 90:
        feedback = f"Good! Knee angle: {knee_angle:.2f} degrees"
        # is_bent_correctly = True
    else:
        feedback = f"Bad! Knee angle: {knee_angle:.2f}degrees"
        # is_bent_correctly = False

    # return feedback, is_bent_correctly
    return feedback, knee_angle

In [16]:
def draw_feedback(image, feedback, state, squat_count, extra_lines=None):
    # 스쿼트 상태 및 횟수 표시
    cv2.putText(
        image,
        f"State: {state}, Squat Count: {squat_count}",
        # (50, 50),  # 텍스트 위치 (x, y)
        # cv2.FONT_HERSHEY_SIMPLEX,
        # 1.0, (0, 255, 0), 2, cv2.LINE_AA
        (30, 50),
        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2, cv2.LINE_AA
    )

    # 피드백 정보 표시 (우측 상단)
    cv2.putText(
        image,
        feedback,
        (30, 90),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.8, (255,0,0), 2, cv2.LINE_AA
    )

    if extra_lines:
        y = 130
        for line in extra_lines:
            cv2.putText(image, line, (30, y),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.6, (0, 255, 255), 2, cv2.LINE_AA)
            y += 35


In [17]:
def resize_image(image, max_width):
    w = image.shape[1]
    if w <= max_width:
        return image
    scale = max_width / w
    h = int(image.shape[0] * scale)
    return cv2.resize(image, (max_width, h))

In [18]:
def process_state(landmarks, state, initial_hip_y, sit_start_time, squat_count):
    # feedback = ""
    extra = []

    left_hip_y = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
    left_knee_y = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y
    left_shoulder_y = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y

    # 처음 자세의 엉덩이 위치 저장
    if initial_hip_y is None:
        initial_hip_y = left_hip_y
        print(f"Initial Hip Y: {initial_hip_y:.3f}")

    # 허리 펴짐 정도 계산
    back_straightness = abs(left_hip_y - left_shoulder_y)

    # 상태 전환 로직
    if state == "START":
        feedback = "Check posture..."
        if back_straightness < BACK_STRAIGHTNESS_THRESHOLD:
            extra.append("Back: Good")
        else:
            extra.append("Back: Straighten!")
        if abs(left_hip_y - left_knee_y) < 0.05:
            state = "SIT"
            sit_start_time = time.time()
            print("State -> SIT")
    elif state == "SIT":
        feedback, knee_angle = check_knee_angle(landmarks)
        extra.append(f"KneeAngle: {knee_angle:.1f}")
        if time.time() - sit_start_time > TIME_LIMIT:
            state = "START"
            feedback = "Timeout. Reset."
            print("Timeout -> START")
        elif left_hip_y < left_knee_y - 0.05 and (left_hip_y <= initial_hip_y + 0.05):
            state = "STAND"
            squat_count += 1
            feedback = f"Squat Count: {squat_count}"
            print(f"스쿼트 횟수: {squat_count}")
    elif state == "STAND":
        if back_straightness < BACK_STRAIGHTNESS_THRESHOLD:
            feedback = "Standing: Good posture"
        else:
            feedback = "Standing: Straighten back"
        if left_hip_y >= left_knee_y - 0.05:
            state = "SIT"
            sit_start_time = time.time()
            feedback = "Transitioning to SIT."
            print("State -> SIT")

    return state, squat_count, feedback, sit_start_time, initial_hip_y, extra

In [19]:
cap = cv2.VideoCapture(0) # 실시간 웹캠 사용

# 전역 변수 초기화
squat_count = 0
initial_hip_y = None
state = "START"
sit_start_time = None

# FPS 측정용
prev_time = time.time()
frame_count = 0
fps = 0.0

with mp_pose.Pose(
    model_complexity=1,  # 0: lite, 1: full, 2: heavy
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
      print("Frame grab failed.")
      break

    # BGR → RGB 변환
    frame.flags.writeable = False
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)
    feedback = ""  # 피드백 초기화
    extra_lines = []

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        state, squat_count, feedback, sit_start_time, initial_hip_y, extra_lines = process_state(landmarks, state, initial_hip_y, sit_start_time, squat_count)

    frame.flags.writeable = True
    frame = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
    if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
    )

    # 피드백 및 상태 정보 표시
    draw_feedback(frame, feedback, state, squat_count, extra_lines)

    # FPS 계산 (0.5초마다 갱신)
    frame_count += 1
    now = time.time()
    if now - prev_time >= 0.5:
      fps = frame_count / (now - prev_time)
      prev_time = now
      frame_count = 0
    cv2.putText(frame, f"FPS: {fps:.1f}", (30, frame.shape[0]-20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2, cv2.LINE_AA)

    # 영상 크기 조정 (원본 비율 유지, 최대 너비 720px로 제한)
    resized_image = resize_image(frame, MAX_WIDTH)

    # Flip 제거 (원본 영상 그대로 표시)
    cv2.imshow('MediaPipe Pose', resized_image)
    key = cv2.waitKey(1) & 0xFF
    if key == 27 or key == ord('q'):
      break

cap.release()
cv2.destroyAllWindows()

Initial Hip Y: 2.223
State -> SIT
스쿼트 횟수: 1
State -> SIT
스쿼트 횟수: 2
State -> SIT
스쿼트 횟수: 3
State -> SIT
스쿼트 횟수: 4


## 임시 피드백 생성

- start
    - 조건 : 초기 상태라서 X
    - 피드백 : 허리 펴짐 정도 계산
- stand
    - 피드백 : 허리 펴짐 정도 계산
    - 조건
        - 올바르게 펴진 자세인지
        - 제일 처음 start 시의 자세만큼 올라왔는지(w 엉덩이 높이)
        - 정해진 시간 이내에 sit -> stand 자세로 변경하였는지(state 사이에 랜드마크 오류 및 사용자가 사라지는 경우 초기부터 시작하기 위함)
- sit
    - 피드백 : 무릎 굽힘 정도 계산
    - 조건
        - 엉덩이가 무릎과 비슷한 높이인지

In [2]:
def check_knee_angle(landmarks):
    """
    무릎 각도를 계산하고, 피드백을 반환합니다.
    """
    left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

    # 무릎 각도 계산
    knee_angle = calculate_angle(left_hip, left_knee, left_ankle)

    # 피드백 생성
    if knee_angle < 90:
        feedback = f"Good! Knee angle: {knee_angle:.2f} degrees"
        is_bent_correctly = True
    else:
        feedback = f"Bad! Knee angle: {knee_angle:.2f}degrees"
        is_bent_correctly = False

    return feedback, is_bent_correctly

In [3]:
def calculate_angle(a, b, c):
    """
    세 점 a, b, c를 기준으로 b를 중심으로 한 각도를 계산합니다.
    """
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ab = a - b
    bc = c - b

    cosine_angle = np.dot(ab, bc) / (np.linalg.norm(ab) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

In [4]:
def draw_feedback(image, feedback, state, squat_count):
    """
    피드백과 상태 정보를 영상에 표시합니다.
    """
    # 스쿼트 상태 및 횟수 표시
    cv2.putText(
        image,
        f"State: {state}, Squat Count: {squat_count}",
        (50, 50),  # 텍스트 위치 (x, y)
        cv2.FONT_HERSHEY_SIMPLEX,
        2,
        (0, 255, 0),  # 초록색
        3,
        cv2.LINE_AA
    )

    # 피드백 정보 표시 (우측 상단)
    cv2.putText(
        image,
        feedback,
        (50, 120),
        # (image.shape[1] - 400, 50),  # 우측 상단에 위치
        cv2.FONT_HERSHEY_SIMPLEX,
        2,
        (255, 0, 0),  # 파란색
        3,
        cv2.LINE_AA
    )

In [5]:
def resize_image(image, max_width):
    original_width = image.shape[1]
    original_height = image.shape[0]
    scale = max_width / original_width
    resized_width = int(original_width * scale)
    resized_height = int(original_height * scale)
    return cv2.resize(image, (resized_width, resized_height))

In [6]:
import math
import time
import cv2
import mediapipe as mp

# MediaPipe 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# 스쿼트 영상 경로
VIDEO_PATH = 'C:\\Users\\mia00\\Desktop\\CD\\resources\\squat.mp4'
MAX_WIDTH = 500  # 영상 최대 너비
TIME_LIMIT = 10  # 시간 제한 (초)
BACK_STRAIGHTNESS_THRESHOLD = 0.15  # 허리 펴짐 기준

In [ ]:
def process_state(landmarks, state, initial_hip_y, sit_start_time, squat_count):
    feedback = ""
    left_hip_y = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
    left_knee_y = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y
    left_shoulder_y = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y

    # 처음 자세의 엉덩이 위치 저장
    if initial_hip_y is None:
        initial_hip_y = left_hip_y
        print(f"Initial Hip Y: {initial_hip_y}")

    # 허리 펴짐 정도 계산
    back_straightness = abs(left_hip_y - left_shoulder_y)

    # 상태 전환 로직
    if state == "START":
        if back_straightness < BACK_STRAIGHTNESS_THRESHOLD:
            feedback = "Good posture while standing!"
        else:
            feedback = "Bad posture! Straighten your back."
        if abs(left_hip_y - left_knee_y) < 0.05:
            state = "SIT"
            sit_start_time = time.time()
            print("State changed to SIT")
    elif state == "SIT":
        feedback, is_bent_correctly = check_knee_angle(landmarks)
        if time.time() - sit_start_time > TIME_LIMIT:
            state = "START"
            feedback = "Time exceeded! Resetting to START."
            print("시간 초과! State changed to START")
        elif left_hip_y < left_knee_y - 0.05 and (left_hip_y <= initial_hip_y + 0.05):
            state = "STAND"
            squat_count += 1
            feedback = f"Squat Count: {squat_count}"
            print(f"스쿼트 횟수: {squat_count}")
    elif state == "STAND":
        if back_straightness < BACK_STRAIGHTNESS_THRESHOLD:
            feedback = "Good posture while standing!"
        else:
            feedback = "Bad posture! Straighten your back."
        if left_hip_y >= left_knee_y - 0.05:
            state = "SIT"
            sit_start_time = time.time()
            feedback = "Transitioning to SIT."
            print("State changed to SIT")

    return state, squat_count, feedback, sit_start_time, initial_hip_y

In [8]:
cap = cv2.VideoCapture(VIDEO_PATH)

# 전역 변수 초기화
squat_count = 0
initial_hip_y = None
state = "START"
sit_start_time = None

with mp_pose.Pose(
    model_complexity=2,  # 0: lite, 1: full, 2: heavy
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Video has ended or failed to load.")
      break

    # BGR → RGB 변환
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    results = pose.process(image)

    feedback = ""  # 피드백 초기화

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        state, squat_count, feedback, sit_start_time, initial_hip_y = process_state(landmarks, state, initial_hip_y, sit_start_time, squat_count)

    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

    # 피드백 및 상태 정보 표시
    draw_feedback(image, feedback, state, squat_count)
    # 영상 크기 조정 (원본 비율 유지, 최대 너비 720px로 제한)
    resized_image = resize_image(image, MAX_WIDTH)

    # Flip 제거 (원본 영상 그대로 표시)
    cv2.imshow('MediaPipe Pose', resized_image)
    if cv2.waitKey(5) & 0xFF == 27:
      break

cap.release()
cv2.destroyAllWindows()

Initial Hip Y: 0.5344144701957703
State changed to SIT


NameError: name 'np' is not defined